Tạo và kết nối đến database của ArangoDB

In [ ]:
from arango import ArangoClient

# Kết nối đến ArangoDB
client = ArangoClient()

# Thông tin đăng nhập
host = "http://localhost:8529"
username = "root"  # Thay bằng tài khoản của bạn
password = "1231"  # Thay bằng mật khẩu của bạn

# Kết nối với server
sys_db = client.db('_system', username=username, password=password)

# Tạo cơ sở dữ liệu mới
db_name = "graph_database"
if not sys_db.has_database(db_name):
    sys_db.create_database(db_name)
    print(f"Database '{db_name}' đã được tạo thành công!")
else:
    print(f"Database '{db_name}' đã tồn tại.")

# Kết nối đến database mới
db = client.db(db_name, username=username, password=password)
print(f"Đã kết nối đến database '{db_name}'.")


Thêm dữ liệu vào database

In [ ]:
from arango import ArangoClient

# Kết nối đến ArangoDB
client = ArangoClient()

# Thông tin kết nối
db_name = "graph_database"
username = "root"
password = "your_password"  # Đổi thành mật khẩu của bạn
host = "http://localhost:8529"

# Kết nối với database
sys_db = client.db('_system', username=username, password=password)

# Tạo database nếu chưa có
if not sys_db.has_database(db_name):
    sys_db.create_database(db_name)

db = client.db(db_name, username=username, password=password)

# Tạo collection cho vertices và edges
if not db.has_collection('vertices'):
    vertices = db.create_collection('vertices')
else:
    vertices = db.collection('vertices')

if not db.has_collection('edges'):
    edges = db.create_collection('edges', edge=True)
else:
    edges = db.collection('edges')

# Đỉnh (vertices)
vertex_keys = ['A', 'B', 'C', 'D', 'E', 'F']
for vertex in vertex_keys:
    if not vertices.has(vertex):
        vertices.insert({'_key': vertex})

# Cạnh (edges) với trọng số
edges_data = [
    ('A', 'B', 3), ('A', 'C', 5), ('A', 'D', 9),
    ('B', 'C', 3), ('B', 'D', 4), ('B', 'E', 7),
    ('C', 'E', 6), ('C', 'D', 2), ('C', 'F', 8),
    ('D', 'E', 2), ('D', 'F', 2),
    ('E', 'F', 5),
]

for from_vertex, to_vertex, weight in edges_data:
    edge_key = f"{from_vertex}_{to_vertex}"
    if not edges.has(edge_key):
        edges.insert({
            '_from': f'vertices/{from_vertex}',
            '_to': f'vertices/{to_vertex}',
            'weight': weight,
            '_key': edge_key
        })

print("Đồ thị đã được thêm thành công vào ArangoDB!")


BFS

In [ ]:
from arango import ArangoClient
from collections import deque

# Kết nối đến ArangoDB
client = ArangoClient()
db = client.db('graph_database', username='root', password='your_password')

# Lấy danh sách các đỉnh và cạnh
vertices = {vertex['_key']: [] for vertex in db.collection('vertices').all()}
for edge in db.collection('edges').all():
    from_vertex = edge['_from'].split('/')[1]
    to_vertex = edge['_to'].split('/')[1]
    vertices[from_vertex].append(to_vertex)
    vertices[to_vertex].append(from_vertex)  # Nếu là đồ thị vô hướng

print("Graph structure:", vertices)



In [ ]:
def bfs(graph, start_vertex):
    visited = set()  # Tập các đỉnh đã thăm
    queue = deque([start_vertex])  # Hàng đợi cho BFS
    visited_order = []  # Thứ tự duyệt các đỉnh

    while queue:
        vertex = queue.popleft()
        if vertex not in visited:
            visited.add(vertex)
            visited_order.append(vertex)
            # Thêm các đỉnh kề chưa thăm vào hàng đợi
            queue.extend(neighbor for neighbor in graph[vertex] if neighbor not in visited)
    
    return visited_order

# Duyệt BFS bắt đầu từ đỉnh
start = 'A'
visited_order = bfs(vertices, start)
print("BFS visited order:", visited_order)

DFS

In [ ]:
from arango import ArangoClient

# Kết nối đến ArangoDB
client = ArangoClient()
db = client.db('graph_database', username='root', password='your_password')

# Tạo đồ thị từ ArangoDB
graph = {vertex['_key']: [] for vertex in db.collection('vertices').all()}

# Lấy các cạnh từ collection 'edges' và xây dựng danh sách kề
for edge in db.collection('edges').all():
    from_vertex = edge['_from'].split('/')[1]  # ID của đỉnh bắt đầu
    to_vertex = edge['_to'].split('/')[1]      # ID của đỉnh kết thúc
    graph[from_vertex].append(to_vertex)

print("Graph structure:", graph)


In [ ]:
def dfs(graph, start_vertex, visited=None):

    if visited is None:
        visited = set()
    visited.add(start_vertex)
    visited_order = [start_vertex]

    # Duyệt tất cả các đỉnh kề của đỉnh hiện tại
    for neighbor in graph[start_vertex]:
        if neighbor not in visited:
            visited_order.extend(dfs(graph, neighbor, visited))

    return visited_order


# Chạy DFS từ đỉnh 'A'
start = 'A'
visited_order = dfs(graph, start)
print("DFS visited order starting from {}: {}".format(start, visited_order))


DIJKSTRA


In [ ]:
from arango import ArangoClient
import heapq

# Kết nối đến ArangoDB
client = ArangoClient()
db = client.db('graph_database', username='root', password='your_password')

# Tạo đồ thị với trọng số
graph = {vertex['_key']: [] for vertex in db.collection('vertices').all()}

# Lấy dữ liệu từ collection 'edges' và xây dựng danh sách kề (adjacency list)
for edge in db.collection('edges').all():
    from_vertex = edge['_from'].split('/')[1]  # Lấy phần ID của vertex
    to_vertex = edge['_to'].split('/')[1]      # Lấy phần ID của vertex
    weight = edge['weight']  # Trọng số của cạnh
    graph[from_vertex].append((to_vertex, weight))
    graph[to_vertex].append((from_vertex, weight))  # Đồ thị vô hướng

print("Graph structure with weights:", graph)


In [ ]:
def dijkstra(graph, start_vertex, end_vertex):
    # Khởi tạo
    distances = {vertex: float('inf') for vertex in graph}  # Khoảng cách từ start_vertex đến mỗi đỉnh
    previous_vertices = {vertex: None for vertex in graph}  # Lưu đỉnh trước đó trên đường đi ngắn nhất
    distances[start_vertex] = 0  # Khoảng cách từ start_vertex đến chính nó là 0
    priority_queue = [(0, start_vertex)]  # Hàng đợi ưu tiên (min-heap)

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        # Nếu đến được đỉnh đích thì dừng
        if current_vertex == end_vertex:
            break

        # Duyệt các đỉnh kề
        for neighbor, weight in graph[current_vertex]:
            distance = current_distance + weight

            # Cập nhật nếu tìm thấy đường đi ngắn hơn
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_vertices[neighbor] = current_vertex
                heapq.heappush(priority_queue, (distance, neighbor))

    # Truy vết đường đi ngắn nhất
    path = []
    current_vertex = end_vertex
    while current_vertex is not None:
        path.insert(0, current_vertex)
        current_vertex = previous_vertices[current_vertex]

    total_distance = distances[end_vertex]
    return path, total_distance

# Chạy Dijkstra từ 'A' đến 'F'
start = 'A'
end = 'F'
shortest_path, total_distance = dijkstra(graph, start, end)

print("Shortest path from {} to {}: {}".format(start, end, shortest_path))
print("Total distance:", total_distance)
